In [1]:
import numpy as np
from scipy.spatial.distance import cdist, pdist, squareform
from tqdm import tqdm_notebook

from tools.data_loading import load_images, load_labels, dummy_code, load_images_resized
from tools.feature_learning import pins_generation
from tools.hog import hog
from tools.kernels import kernel_matrix
from tools.optimization import find_f
from tools.prediction import pred
from tools.process_images import process_images
from tools.quantization import kmeans
from tools.submission import labels_to_csv
from tools.visualization import imshow, dump_as_png, reshape_as_images

# Data loading

In [2]:
X_train = load_images_resized(type="train")
n_train = X_train.shape[0]
Y_labels_train = load_labels()
Y_train = dummy_code(Y_labels_train)
n_classes = Y_train.shape[1]
X_train = X_train - X_train.min(axis=0)
X_train = X_train / X_train.max(axis=0)

In [3]:
X_train2 = np.zeros((5000,63,63,3))
for i in range(3):
    X_train2[:,:,:,i] = X_train
image_list = X_train2

# Visual features

In [4]:
filter_sigma=0.1
filter_shape=5
hog_cell_size=8
disc_grid=16

In [5]:
n_images = image_list.shape[0]

In [6]:
hog_list = []
for i in tqdm_notebook(range(n_images), desc="Images"):
    image = image_list[i,:,:,:]
    hog_list.append(
        hog(
            image,
            filter_sigma=filter_sigma,
            filter_shape=filter_shape,
            hog_cell_size=hog_cell_size,
            disc_grid=disc_grid))
    
n_features = hog_list[0].size
X_hog = np.array(hog_list).reshape((n_images, n_features)) 


Widget Javascript not detected.  It may not be installed or enabled properly.


# Data separation

In [32]:
indices = np.random.permutation(X_hog.shape[0])
training_idx, test_idx = indices[:int(0.9*n_train)], indices[int(0.9*n_train):]

X_sample = X_hog[training_idx, :]
n_sample = X_sample.shape[0]
Y_sample = Y_train[training_idx,:]
Y_labels_sample = Y_labels_train[training_idx]

X_test = X_hog[test_idx, :]
n_test = X_test.shape[0]
Y_labels_test = Y_labels_train[test_idx]

## Pre-computing distances

In [33]:
train_dist = squareform(pdist(X_sample, "euclidean"))
train_test_dist = cdist(X_sample[:500], X_sample, "euclidean")
test_dist = cdist(X_test, X_sample, "euclidean")

## Pre-computing inner products

In [34]:
train_inner = X_sample.dot(X_sample.T)
train_test_inner = X_sample[:500].dot(X_sample.T)
test_inner = X_test.dot(X_sample.T)

# Training

In [10]:
kernel_type = "polynomial"
classifier_type = "svm"
degree = 4
constant = 5
lamb = 1

In [60]:
kernel_type="rbf"
sigma = 15.5
lamb = 1e-8

## Kernel choice

In [61]:
K_sample = kernel_matrix(
    X_sample,
    kernel_type=kernel_type,
    degree=degree,
    constant=constant,
    sigma=sigma,
    dist=train_dist,
    inner=train_inner)

In [62]:
K_sample.shape

(4500, 4500)

## Classifier choice

In [64]:
alpha = np.zeros((n_classes, n_sample))
for dig in tqdm_notebook(range(n_classes), desc="Classes"):
    alpha[dig, :] = find_f(K_sample, Y_sample[:, dig],
                           prob_type=classifier_type, lamb=lamb, n_iter=100000)

Widget Javascript not detected.  It may not be installed or enabled properly.


# Evaluation

### Training error

In [65]:
Y_pred_train = pred(
    X_sample, X_sample[:500], alpha,
    kernel_type=kernel_type,
    degree=degree,
    constant=constant,
    sigma=sigma,
    dist=train_test_dist,
    inner=train_test_inner)


Y_labels_pred_train = np.argmax(Y_pred_train, axis=1)
prec = np.mean(Y_labels_pred_train == Y_labels_sample[:500])
print("The precision on the train set is of {}".format(prec))

The precision on the train set is of 1.0


### Testing error

In [66]:
Y_pred = pred(
    X_sample, X_test, alpha,
    kernel_type=kernel_type,
    degree=degree,
    constant=constant,
    sigma=sigma,
    dist=test_dist,
    inner=test_inner)


Y_labels_pred = np.argmax(Y_pred, axis=1)
prec = np.mean(Y_labels_pred == Y_labels_test)
print("The precision on the test set is of {}".format(prec))

The precision on the test set is of 0.584


## Grid search from hell

### For polynomial

In [68]:
kernel_type = "polynomial"
degrees = [4, 5, 6]
n_degrees = len(degrees)
constants = [1, 5, 50, 100]
n_constants = len(constants)
lambs = [1e-9, 1e-5, 0.1, 1]
n_lambs = len(lambs)
train_prec = np.zeros((n_degrees, n_constants, n_lambs))
test_prec = np.zeros((n_degrees, n_constants, n_lambs))
for d in tqdm_notebook(range(n_degrees), desc="Degrees"):
    for c in tqdm_notebook(range(n_constants), desc="Constants"):
        # Kernel matrix computation
        K_sample = kernel_matrix(
            X_sample, kernel_type=kernel_type, degree=degrees[d], constant=constants[c], inner=train_inner)
        for l in tqdm_notebook(range(n_lambs), desc="Lambdas"):
            # Coordinate ascent
            alpha = np.zeros((n_classes, n_sample))
            for dig in tqdm_notebook(range(n_classes), desc="Classes"):
                alpha[dig, :] = find_f(K_sample, Y_sample[:, dig],
                                       prob_type=classifier_type, lamb=lambs[l], n_iter=100000)
            # Precision on train set
            Y_pred_train = pred(
                X_sample, X_sample[:500], alpha,
                kernel_type=kernel_type,
                degree=degrees[d],
                constant=constants[c],
                inner=train_test_inner)

            Y_labels_pred_train = np.argmax(Y_pred_train, axis=1)
            train_prec[d, c, l] = np.mean(Y_labels_pred_train == Y_labels_sample[:500])
            # Precision on test set
            Y_pred= pred(
                X_sample, X_test, alpha,
                kernel_type=kernel_type,
                degree=degrees[d],
                constant=constants[c],
                inner=test_inner)

            Y_labels_pred = np.argmax(Y_pred, axis=1)
            test_prec[d, c, l] = np.mean(Y_labels_pred == Y_labels_test)

Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


...

Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


In [69]:
test_prec

array([[[ 0.552,  0.552,  0.552,  0.552],
        [ 0.552,  0.552,  0.554,  0.55 ],
        [ 0.562,  0.562,  0.562,  0.562],
        [ 0.562,  0.562,  0.562,  0.562]],

       [[ 0.562,  0.562,  0.562,  0.562],
        [ 0.562,  0.562,  0.562,  0.562],
        [ 0.572,  0.572,  0.572,  0.572],
        [ 0.564,  0.564,  0.564,  0.564]],

       [[ 0.564,  0.564,  0.564,  0.564],
        [ 0.562,  0.562,  0.562,  0.562],
        [ 0.568,  0.568,  0.568,  0.568],
        [ 0.56 ,  0.56 ,  0.56 ,  0.56 ]]])

In [70]:
train_prec

array([[[ 1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.]],

       [[ 1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.]],

       [[ 1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.]]])

### For RBF

In [57]:
kernel_type="rbf"
sigmas = [15.5, 16, 16.5]
n_sigmas = len(sigmas)
lambs = [1e-9, 1e-8, 1e-7]
n_lambs = len(lambs)
train_prec_rbf = np.zeros((n_sigmas, n_lambs))
test_prec_rbf = np.zeros((n_sigmas, n_lambs))
for s in tqdm_notebook(range(n_sigmas), desc="Sigmas"):
    # Kernel matrix computation
    K_sample = kernel_matrix(X_sample, kernel_type=kernel_type, sigma=sigmas[s], dist=train_dist)
    for l in tqdm_notebook(range(n_lambs), desc="Lambdas"):
        # Coordinate ascent
        alpha = np.zeros((n_classes, n_sample))
        for dig in tqdm_notebook(range(n_classes), desc="Classes"):
            alpha[dig, :] = find_f(K_sample, Y_sample[:, dig],
                                   prob_type=classifier_type, lamb=lambs[l], n_iter=50000)
        # Precision on train set
        Y_pred_train= pred(X_sample, X_sample[:500], alpha,
                              kernel_type=kernel_type, sigma=sigmas[s], dist=train_test_dist)
        Y_labels_pred_train = np.argmax(Y_pred_train, axis=1)
        train_prec_rbf[s, l] = np.mean(Y_labels_pred_train == Y_labels_sample[:500])
        # Precision on test set
        Y_pred = pred(X_sample, X_test, alpha,
                              kernel_type=kernel_type, sigma=sigmas[s], dist=test_dist)


        Y_labels_pred = np.argmax(Y_pred, axis=1)
        test_prec_rbf[s, l] = np.mean(Y_labels_pred == Y_labels_test)

Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


...

Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


In [58]:
test_prec_rbf

array([[ 0.586,  0.588,  0.586],
       [ 0.584,  0.58 ,  0.578],
       [ 0.574,  0.582,  0.586]])

In [59]:
train_prec_rbf

array([[ 1.,  1.,  1.],
       [ 1.,  1.,  1.],
       [ 1.,  1.,  1.]])

# Prediction

In [ ]:
X_eval = load_images_resized(type="test")
X_eval = X_eval - X_eval.min(axis=0)
X_eval = X_eval / X_eval.max(axis=0)
n_eval = X_eval.shape[0]
X_eval2 = np.zeros((2000,63,63,3))
for i in range(3):
    X_eval2[:,:,:,i] = X_eval
image_list_eval = X_eval2

# Visual features for submission
hog_list_eval = []
for i in tqdm_notebook(range(n_eval), desc="Submission images"):
    image = image_list_eval[i,:,:,:]
    hog_list_eval.append(
        hog(
            image,
            filter_sigma=filter_sigma,
            filter_shape=filter_shape,
            hog_cell_size=hog_cell_size,
            disc_grid=disc_grid))
    
n_features = hog_list_eval[0].size
X_hog_eval = np.array(hog_list_eval).reshape((n_eval, n_features)) 

In [67]:
Y_eval = pred(X_sample, X_hog_eval, alpha,
                      kernel_type=kernel_type, degree=degree, constant=constant, sigma=sigma)


Y_labels_eval = np.argmax(Y_eval, axis=1)


# Submission
labels_to_csv(Y_labels_eval, kernel=kernel_type, algo="svm_hog",user="Zac_big_images")